<a href="http://agrum.org" target="blank"><img src="http://agrum.gitlab.io/theme/img/logoAgrum.png" align="left" style="height:100px"/></a><a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This pyAgrum's notebook is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

from pyAgrum.lib._utils.oslike import head

import os
#the bases will be saved in "out/*.csv"
EMnomissing=os.path.join("out","EM_nomissing.csv")
EMmissing=os.path.join("out","EM_missing.csv")

### generating data with missing values (at random)

In [2]:
src=gum.fastBN("A->B<-C->D->E<-B;D->F")
gum.generateCSV(src,EMnomissing,5000,random_order=False)
src

(gum::BayesNet<double>@0x562d1f10a6f0) BN{nodes: 6, arcs: 6, domainSize: 64, parameters: 28, compression ratio: 56% }

In [3]:
import pandas as pd
import numpy as np

def add_missing(src,dst,proba):
  df=pd.read_csv(src)
  mask=np.random.choice([True, False], size=df.shape,p=[proba,1-proba])
  df.mask(mask).to_csv(dst,na_rep='?',index=False,float_format='%.0f')

gum.generateCSV(src,EMnomissing,5000,random_order=False)
add_missing(EMnomissing,EMmissing,proba=0.1)

In [4]:
print("No missing")
head(EMnomissing)
print("Missing")
head(EMmissing)

No missing
A,B,C,D,E,F
0,0,1,0,0,0
0,0,1,0,0,0
0,0,1,1,1,1
0,1,1,0,1,0
0,1,1,0,1,0
0,0,1,1,1,1
0,0,1,0,0,1
0,1,1,0,0,1
0,0,1,0,0,1

Missing
A,B,C,D,E,F
0,0,?,0,0,0
0,?,1,0,0,0
0,0,1,1,1,1
0,1,1,0,1,0
0,1,1,0,1,0
?,0,1,1,1,?
0,0,1,0,0,1
0,1,?,0,0,1
0,0,1,0,0,1



### learning with missing data

In [5]:
learner = gum.BNLearner(EMmissing, ["?"])
print(f"Missing values in {EMmissing} : {learner.hasMissingValues()}")

Missing values in out/EM_missing.csv : True


In [6]:
# this will fail : missing data !
# learner.learnParameters(src.dag())

In [7]:
learner.useEM(1e-3)
learner.useAprioriSmoothing()
bn=learner.learnParameters(src.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(src),gnb.getInference(bn))

# iterations : 6


### learning with smaller error (and no smoothing)

In [8]:
learner = gum.BNLearner(EMmissing, ["?"])
learner.setVerbosity(True)
learner.useEM(1e-8)
bn2=learner.learnParameters(src.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(src),gnb.getInference(bn2))

# iterations : 14


In [9]:
print(learner.history())

(0.4844705135018414, 0.12254386287350393, 0.02881927590559244, 0.006518373351415568, 0.0014766602503282563, 0.00033760147346458935, 7.792254188970786e-05, 1.8137946953139796e-05, 4.252326973138202e-06, 1.0029177762405674e-06, 2.3771625331870856e-07, 5.6576010292701314e-08, 1.3510598398226302e-08, 3.2354121022972397e-09)
